# Estudio sobre contaminación mundial

### Análisis de la huella ecológica mundial

Las National Footprint Accounts (NFA) en asociacón de la  Global Footprint Network (www.footprintnetwork.org) combinan cada año más de 30 conjuntos de datos para calcular la Huella Ecológica y la biocapacidad de los países de todo el mundo en más de 50 años.

El objetivo de este compromiso es arrojar resultados de cuánta área se requiere para proporcionar los servicios ecológicos (regeneración de recursos y asimilación de desechos) consumidos por la humanidad ("Huella ecológica"), y cuánta área biológicamente productiva existe para proporcionar estos servicios ecológicos ( "biocapacidad") en cada año. Con ambos valores en la mano, podemos evaluar la sostenibilidad general de los países de todo el mundo y comprender mejor la necesidad colectiva de que la humanidad reduzca su impacto en la naturaleza.

En este estudio vamos a intentar estudiar y predecir la capacidad por hectaréa de biocapacidad y la huella ecológica de estas, en el estudio biocapTotGHA y EFConsTotGHA. También intentaremos hacer estudios de esto mismo pero sobre la poulicón sobre carbón.

Estos datos han sido obtenidos de: https://www.kaggle.com/footprintnetwork/national-footprint-accounts-2018

### Creación del dataset

In [4]:
from pyspark.sql import SparkSession
import os
import sys
from pyspark import SparkContext
from pyspark import SparkConf

spark = SparkSession \
    .builder \
    .appName("Entrega - data") \
    .getOrCreate()

Primero vamos a importar el dataframe de ejemplo, la integración directa con kaggle es bastante compleja y por eso hemos optado por importar el dataset a githubusercontent ya que este es relativamente pequeño

In [6]:
%sh
cd /tmp

#  Removemos el fichero antiguo 
if [ -e /tmp/NFA2018.csv]
then
    rm -f /NFA2018.csv
fi

# Descargamos el fichero 
wget https://raw.githubusercontent.com/davidcerezal/deep-learning-examples/master/NFA2018.csv

/bin/bash: line 3: [: missing `]'
--2019-06-30 11:15:08-- https://raw.githubusercontent.com/davidcerezal/deep-learning-examples/master/NFA2018.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11857080 (11M) [text/plain]
Saving to: ‘NFA2018.csv.5’

 0K .......... .......... .......... .......... .......... 0% 3.29M 3s
 50K .......... .......... .......... .......... .......... 0% 6.54M 3s
 100K .......... .......... .......... .......... .......... 1% 142M 2s
 150K .......... .......... .......... .......... .......... 1% 6.46M 2s
 200K .......... .......... .......... .......... .......... 2% 6.46M 2s
 250K .......... .......... .......... .......... .......... 2% 163M 1s
 300K .......... .......... .......... .......... .......... 3% 151M 1s
 350K .......... .......... .......... .......... .......... 3% 150M 1s
 400K .......... .......... .......... .......... .......... 3% 6.78M 1s
 450K .......... .......... .......... .......... .......... 4% 159M 1s
 500K .......... .......... .......... .......... .......... 4% 148M 1s
 550K .......... .......... .......... .......... .......... 5% 136M 1s
 600K .......... .......... .......... .......... .......... 5% 167M 1s
 650K .......... .......... .......... .......... .......... 6% 165M 1s
 700K .......... .......... .......... .......... .......... 6% 172M 1s
 750K .......... .......... .......... .......... .......... 6% 7.42M 1s
 800K .......... .......... .......... .......... .......... 7% 25.7M 1s
 850K .......... .......... .......... .......... .......... 7% 26.3M 1s
 900K .......... .......... .......... .......... .......... 8% 52.5M 1s
 950K .......... .......... .......... .......... .......... 8% 134M 1s
 1000K .......... .......... .......... .......... .......... 9% 166M 1s
 1050K .......... .......... .......... .......... .......... 9% 45.3M 1s
 1100K .......... .......... .......... .......... .......... 9% 167M 1s
 1150K .......... .......... .......... .......... .......... 10% 95.4M 1s
 1200K .......... .......... .......... .......... .......... 10% 43.7M 1s
 1250K .......... .......... .......... .......... .......... 11% 156M 0s
 1300K .......... .......... .......... .......... .......... 11% 162M 0s
 1350K .......... .......... .......... .......... .......... 12% 139M 0s
 1400K .......... .......... .......... .......... .......... 12% 164M 0s
 1450K .......... .......... .......... .......... .......... 12% 165M 0s
 1500K .......... .......... .......... .......... .......... 13% 167M 0s
 1550K .......... .......... .......... .......... .......... 13% 145M 0s
 1600K .......... .......... .......... .......... .......... 14% 165M 0s
 1650K .......... .......... .......... .......... .......... 14% 157M 0s
 1700K .......... .......... .......... .......... .......... 15% 161M 0s
 1750K .......... .......... .......... .......... .......... 15% 128M 0s
 1800K .......... .......... .......... .......... .......... 15% 14.3M 0s
 1850K .......... .......... .......... .......... .......... 16% 10.2M 0s
 1900K .......... .......... .......... .......... .......... 16% 7.80M 0s
 1950K .......... .......... .......... .......... .......... 17% 21.9M 0s
 2000K .......... .......... .......... .......... .......... 17% 22.6M 0s
 2050K .......... .......... .......... .......... .......... 18% 24.4M 0s
 2100K .......... .......... .......... .......... .......... 18% 23.8M 0s
 2150K .......... .......... .......... .......... .......... 18% 19.9M 0s
 2200K .......... .......... .......... .......... .......... 19% 25.0M 0s
 2250K .......... .......... .......... .......... .......... 19% 23.7M 0s
 2300K .......... .......... .......... .......... .......... 20% 67.9M 0s
 2350K .......... .......... .......... .......... .......... 20% 145M 0s
 2400K .......... .......... 

Ahora vamos a convertir el dataset a un dataframe, limipiaremos también algunas variable redudantes.

In [8]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

dbutils.fs.cp("file:///tmp/NFA2018.csv", "/tmp")

patentData = sqlContext.read.format('csv').option("header", 'true').load("/tmp/NFA2018.csv")

patentData.cache()

patentData = patentData.drop("crop_land")
patentData = patentData.drop("grazing_land")
patentData = patentData.drop("forest_land")
patentData = patentData.drop("fishing_ground")
patentData = patentData.drop("built_up_land")

#Filter the bio Capacity and foot print frames, to join later
patentData_bio = patentData.filter("record == 'BiocapTotGHA'").withColumnRenamed("total","BiocapTotGHA").drop('record').drop('carbon').drop('Percapita GDP (2010 USD)').withColumnRenamed("ISO alpha-3 code","ISO")
patentData_foot = patentData.filter("record == 'EFConsTotGHA'").withColumnRenamed("total","EFConsTotGHA").drop('record').drop('ISO alpha-3 code').drop('Percapita GDP (2010 USD)')

#Join and clean up of duplicated columns
patentData = patentData_bio.join(patentData_foot,(patentData_bio.country == patentData_foot.country) & (patentData_bio.year == patentData_foot.year)).drop(patentData_foot.country).drop(patentData_foot.UN_region).drop(patentData_foot.UN_subregion).drop(patentData_foot.year).drop(patentData_foot.population)

#World data
world_data = patentData.filter(patentData.country == 'World')

#countries Data
patentData = patentData.filter(patentData.country != 'World')

patentData = patentData.withColumn("year_c", patentData.year.cast('double')).drop('year')
patentData = patentData.withColumn("population_c", patentData.population.cast('double')).drop('population')
patentData = patentData.withColumn("BiocapTotGHA_c", patentData.BiocapTotGHA.cast('double')).drop('BiocapTotGHA')
patentData = patentData.withColumn("EFConsTotGHA_c", patentData.EFConsTotGHA.cast('double')).drop('EFConsTotGHA')
patentData = patentData.withColumn("carbon_c", patentData.carbon.cast('double')).drop('carbon')

display(patentData)


DataFrame[ISO: string, country: string, UN_region: string, UN_subregion: string, year_c: double, population_c: double, BiocapTotGHA_c: double, EFConsTotGHA_c: double, carbon_c: double]

### Conocer el dataset
Ahora vamos a conoces un poco nuestro, dataset, ver quien es el mas contaminante o el mas bioético.

In [10]:
#Podemos hacer consultas sobre el desarrollo mundial
display(world_data.select( "year", "BiocapTotGHA", "EFConsTotGHA", "carbon", "population"))

#BiocapTotGHA = Total biocapacity in global hectares (gha)
#EFConsTotGHA = Total Ecological Footprint of consumption in global hectares (gha)

#Podemos observar rápidamente que hemos superado claramente nuestra capacidad de producción 

DataFrame[year: string, BiocapTotGHA: string, EFConsTotGHA: string, carbon: string, population: string]

In [11]:
#Podemos hacer consultas tan geniales como estas
last_year_data = patentData.filter('year_c = 2014')
display(last_year_data.select('ISO', 'BiocapTotGHA_c'))

#Total biocapacity in global hectares (gha)

ISO,BiocapTotGHA_c
ARM,2362550.102
AFG,1.594268587E7
ALB,3038634.272
DZA,2.108411419E7
AGO,5.548912623E7
ATG,84838.93322
ARG,2.883548292E8
AUS,3.130206603E8
AUT,2.567365936E7
BHS,3577704.445


In [12]:
#Total Ecological Footprint of consumption in global hectares (gha)
display(last_year_data.select('ISO', 'EFConsTotGHA_c'))

ISO,EFConsTotGHA_c
ARM,6068198.904
AFG,2.442465511E7
ALB,6188638.631
DZA,9.53405725E7
AGO,3.77676048E7
ATG,380368.67
ARG,1.585845213E8
AUS,1.626578588E8
AUT,5.00490291E7
BHS,1847637.762


In [13]:
#Podemos filtar también por aquellos que no consumen más alla de su capacidad o al contrario
last_year_data.createOrReplaceTempView("last_year_data")
less_countries = spark.sql("""SELECT country, BiocapTotGHA_c - EFConsTotGHA_c as left_cosumption FROM last_year_data WHERE BiocapTotGHA_c > EFConsTotGHA_c ORDER BY left_cosumption DESC""")
display(less_countries)

country,left_cosumption
Brazil,1.1900328306E9
Canada,2.5622425330000007E8
Russian Federation,1.8768387669999993E8
Australia,1.503628015E8
"Congo, Democratic Republic of",1.4420442416E8
Bolivia,1.4167006907E8
Argentina,1.2977030789999998E8
Colombia,8.576790297999999E7
Guyana,5.0363366009E7
Peru,4.627369394E7


In [14]:
less_countries = patentData.filter("country = 'Brazil' or country = 'Canada' or country = 'Russian Federation' or country = 'Australia'")
less_countries = less_countries.withColumn('left_comsuption', less_countries.BiocapTotGHA_c - less_countries.EFConsTotGHA_c)
display(less_countries.select('country', 'year_c', 'left_comsuption'))

DataFrame[country: string, year_c: double, left_comsuption: double]

In [15]:
last_year_data.createOrReplaceTempView("last_year_data")
higher_countries = spark.sql("""SELECT country, EFConsTotGHA_c - BiocapTotGHA_c as required_cosumption FROM last_year_data WHERE BiocapTotGHA_c < EFConsTotGHA_c ORDER BY required_cosumption DESC""")
display(higher_countries)

country,required_cosumption
China,3.83018248E9
United States of America,1.527696779E9
India,8.651361629E8
Japan,5.2489409269000006E8
Germany,2.626606777E8
"Korea, Republic of",2.5686862019000003E8
United Kingdom,2.3121720085E8
"Iran, Islamic Republic of",2.0699643764E8
Italy,2.0070719809E8
Saudi Arabia,1.7262900557E8


In [16]:
most_countries = patentData.filter("country = 'China' or country = 'United States of America' or country = 'India' or country = 'Japan'")
most_countries = most_countries.withColumn('req_comsuption', most_countries.EFConsTotGHA_c -  most_countries.BiocapTotGHA_c)
display(most_countries.select('country', 'year_c', 'req_comsuption'))

country,year_c,req_comsuption
India,1961.0,5.894979280000001E7
India,1962.0,6.441964449999997E7
India,1963.0,7.059525349999997E7
India,1964.0,7.1203838E7
India,1965.0,8.127604889999998E7
India,1966.0,9.068650539999998E7
India,1967.0,9.23241889E7
India,1968.0,9.318768589999998E7
India,1969.0,9.38345011E7
India,1970.0,9.719847569999999E7


In [17]:
#Podemos definir udf que nos pueden ayudar mucho como por ejemplo en el siguiente caso
spark.udf.register("calculate_percentage", lambda biocap, req: int(((biocap - req)/req*100)) if biocap > req else int(0-((req - biocap)/biocap*100)))

#Países menos agresivos en sus emisiones
percetages = spark.sql("""SELECT country, CONCAT(CAST(calculate_percentage(BiocapTotGHA_c, EFConsTotGHA_c) as int), '%') as percetage FROM last_year_data ORDER BY CAST(calculate_percentage(BiocapTotGHA_c, EFConsTotGHA_c) as int) DESC""")
display(percetages)

country,percetage
French Guiana,3864%
Suriname,2326%
Guyana,2295%
Gabon,846%
Congo,763%
Central African Republic,554%
Bolivia,436%
"Congo, Democratic Republic of",254%
Uruguay,246%
Namibia,212%


In [18]:
#Países más agresivos en sus emisiones
percetages = spark.sql("""SELECT country, CONCAT(CAST(calculate_percentage(BiocapTotGHA_c, EFConsTotGHA_c) as int), '%') as real_percentage FROM last_year_data ORDER BY CAST(calculate_percentage(BiocapTotGHA_c, EFConsTotGHA_c) as int) ASC""")
display(percetages)

country,real_percentage
Singapore,-9886%
Bermuda,-4810%
Réunion,-2817%
Barbados,-2067%
Cayman Islands,-1673%
United Arab Emirates,-1651%
Israel,-1644%
Bahrain,-1529%
Saudi Arabia,-1351%
Cyprus,-1262%


### Creación del modelo

In [20]:
#Vamos a eliminar algunas variables por ser redundantes
patentData = patentData.na.fill(0)
patentData = patentData.drop('country')

#La variable que vamos a intentar predecir
patentData = patentData.withColumnRenamed('BiocapTotGHA_c', 'label')
patentData = patentData.fillna({'label': 0})


#primero podemos empezar creando la partición de train y test, en este caso para conservar las variables de tiempo en las predicciones no lo vamos a hacer aleatoriamente y lo filtraremos por año, dejando los tres últimos para test. 
patentData_train = patentData.filter(patentData.year_c < 2011)
patentData_test = patentData.filter(patentData.year_c > 2011)
display(patentData)

DataFrame[ISO: string, UN_region: string, UN_subregion: string, year_c: double, population_c: double, label: double, EFConsTotGHA_c: double, carbon_c: double]

Preparación de los datos para el modelo. Para ellos creamos diferentes stages, con OneHotEncoderEstimator, StringIndexer y VectorAssembler.

In [22]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

stages = []
categoricalColumns = ['ISO', 'UN_region', 'UN_subregion']
numericCols = ['year_c', 'population_c', 'EFConsTotGHA_c', 'carbon_c']

for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
assembler.setHandleInvalid("keep")
stages += [assembler]

print(stages)


[StringIndexer_f0563309fe56, OneHotEncoderEstimator_6c4cbb173924, StringIndexer_45303dda70b8, OneHotEncoderEstimator_06ac08112b9a, StringIndexer_2670bc279239, OneHotEncoderEstimator_f1935a863ba2, VectorAssembler_761ba5bfbb59]

Primero vamos a empezar con un Random Forest Regressor

In [24]:
rf = RandomForestRegressor(featuresCol = 'features', labelCol = 'label')

stages += [rf]
pipeline = Pipeline(stages=stages)

#Seleccionamos los hiperparámetros a buscar
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [int(x) for x in np.linspace(start = 10, stop = 50, num = 3)]) \
    .addGrid(rf.maxDepth, [int(x) for x in np.linspace(start = 5, stop = 25, num = 3)]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(),
                          numFolds=3)

cvModel = crossval.fit(patentData_train)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2218260910706559> in <module> () 
 12 numFolds=3)
 13 
 ---> 14 cvModel = crossval . fit ( patentData_train ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/tuning.py in _fit (self, dataset) 
 305 
 306 tasks = _parallelFitTasks ( est , train , eva , validation , epm , collectSubModelsParam ) 
 --> 307 for j , metric , subModel in pool . imap_unordered ( lambda f : f ( ) , tasks ) : 
 308 metrics [ j ] += ( metric / nFolds ) 
 309 metrics_all [ i ] [ j ] = metric 

 /usr/lib/python3.5/multiprocessing/pool.py in next (self, timeout) 
 693 if success : 
 694 return value 
 --> 695 raise value 
 696 
 697 __next__ = next # XXX 

 /usr/lib/python3.5/multiprocessing/pool.py in worker (inqueue, outqueue, initializer, initargs, maxtasks, wrap_exception) 
 117 job , i , func , args , kwds = task 
 118 try : 
 --> 119 result = ( True , func ( * args , ** kwds ) ) 
 120 except Exception as e : 
 121 if wrap_exception : 

 /databricks/spark/python/pyspark/ml/tuning.py in <lambda> (f) 
 305 
 306 tasks = _parallelFitTasks ( est , train , eva , validation , epm , collectSubModelsParam ) 
 --> 307 for j , metric , subModel in pool . imap_unordered ( lambda f : f ( ) , tasks ) : 
 308 metrics [ j ] += ( metric / nFolds ) 
 309 metrics_all [ i ] [ j ] = metric 

 /databricks/spark/python/pyspark/ml/tuning.py in singleTask () 
 52 
 53 def singleTask ( ) : 
 ---> 54 index , model = next ( modelIter ) 
 55 metric = eva . evaluate ( model . transform ( validation , epm [ index ] ) ) 
 56 return index , metric , model if collectSubModel else None 

 /databricks/spark/python/pyspark/ml/base.py in __next__ (self) 
 60 raise StopIteration ( "No models remaining." ) 
 61 self . counter += 1 
 ---> 62 return index , self . fitSingleModel ( index ) 
 63 
 64 def next ( self ) : 

 /databricks/spark/python/pyspark/ml/base.py in fitSingleModel (index) 
 104 
 105 def fitSingleModel ( index ) : 
 --> 106 return estimator . fit ( dataset , paramMaps [ index ] ) 
 107 
 108 return _FitMultipleIterator ( fitSingleModel , len ( paramMaps ) ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 128 elif isinstance ( params , dict ) : 
 129 if params : 
 --> 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 132 return self . _fit ( dataset ) 

 /databricks/spark/python/pyspark/ml/pipeline.py in _fit (self, dataset) 
 107 dataset = stage . transform ( dataset ) 
 108 else : # must be an Estimator 
 --> 109 model = stage . fit ( dataset ) 
 110 transformers . append ( model ) 
 111 if i < indexOfLastEstimator : 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 293 
 294 def _fit ( self , dataset ) : 
 --> 295 java_model = self . _fit_java ( dataset ) 
 296 model = self . _create_model ( java_model ) 
 297 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 290 """
 291 self . _transfer_params_to_java ( ) 
 --> 292 return self . _java_obj . fit ( dataset . _jdf ) 
 293 
 294 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_cl

## Evaluación del modelo

vamos a evaluar ahora a los resultados de este.

In [26]:
predictions = cvModel.transform(patentData_test)

predictions = cvModel.transform(patentData_test)
predictions.select('ISO','year_c', 'population_c', 'BiocapTotGHA_c', 'EFConsTotGHA_c', 'carbon_c').show(10)

+---+------+------------+--------------+--------------+-----------+
ISO|year_c|population_c|BiocapTotGHA_c|EFConsTotGHA_c| carbon_c|
+---+------+------------+--------------+--------------+-----------+
ARM|2012.0| 2978000.0| 2230728.29| 6371810.678|2601443.533|
ARM|2013.0| 2992000.0| 2351551.194| 6083714.761|2584958.796|
ARM|2014.0| 3006000.0| 2362550.102| 6068198.904|2560362.241|
AFG|2012.0| 2.9727E7| 1.541892059E7| 2.406439112E7|5032811.331|
AFG|2013.0| 3.0682E7| 1.558024351E7| 2.425068459E7|5112870.611|
AFG|2014.0| 3.1628E7| 1.594268587E7| 2.442465511E7|4898568.052|
ALB|2012.0| 2881000.0| 3084853.738| 6426738.013|2790481.735|
ALB|2013.0| 2883000.0| 3009000.371| 6312021.919|2775526.436|
ALB|2014.0| 2890000.0| 3038634.272| 6188638.631|2592726.604|
DZA|2012.0| 3.7439E7| 2.34017506E7| 9.049694707E7| 0.0|
+---+------+------------+--------------+--------------+-----------+
only showing top 10 rows

In [27]:
import numpy as np
import matplotlib.pyplot as plt

evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

rmse = evaluator.evaluate(predictions)

display(rfResult)

ISO UN_region UN_subregion year_c \
0 ARM Asia Western Asia 2012.0 
1 ARM Asia Western Asia 2013.0 
2 ARM Asia Western Asia 2014.0 
3 AFG Asia Southern Asia 2012.0 
4 AFG Asia Southern Asia 2013.0 
5 AFG Asia Southern Asia 2014.0 
6 ALB Europe Southern Europe 2012.0 
7 ALB Europe Southern Europe 2013.0 
8 ALB Europe Southern Europe 2014.0 
9 DZA Africa Northern Africa 2012.0 
10 DZA Africa Northern Africa 2013.0 
11 DZA Africa Northern Africa 2014.0 
12 AGO Africa Middle Africa 2012.0 
13 AGO Africa Middle Africa 2013.0 
14 AGO Africa Middle Africa 2014.0 
15 ATG Latin America and the Caribbean Caribbean 2012.0 
16 ATG Latin America and the Caribbean Caribbean 2013.0 
17 ATG Latin America and the Caribbean Caribbean 2014.0 
18 ARG Latin America and the Caribbean South America 2012.0 
19 ARG Latin America and the Caribbean South America 2013.0 
20 ARG Latin America and the Caribbean South America 2014.0 
21 AUS Oceania Australia and New Zealand 2012.0 
22 AUS Oceania Australia and New Zealand 2013.0 
23 AUS Oceania Australia and New Zealand 2014.0 
24 AUT Europe Western Europe 2012.0 
25 AUT Europe Western Europe 2013.0 
26 AUT Europe Western Europe 2014.0 
27 BHS Latin America and the Caribbean Caribbean 2012.0 
28 BHS Latin America and the Caribbean Caribbean 2013.0 
29 BHS Latin America and the Caribbean Caribbean 2014.0 
.. ... ... ... ... 
524 YEM Asia Western Asia 2012.0 
525 YEM Asia Western Asia 2013.0 
526 YEM Asia Western Asia 2014.0 
527 COD Africa Middle Africa 2012.0 
528 COD Africa Middle Africa 2013.0 
529 COD Africa Middle Africa 2014.0 
530 ZMB Africa Eastern Africa 2012.0 
531 ZMB Africa Eastern Africa 2013.0 
532 ZMB Africa Eastern Africa 2014.0 
533 BEL Europe Western Europe 2012.0 
534 BEL Europe Western Europe 2013.0 
535 BEL Europe Western Europe 2014.0 
536 LUX Europe Western Europe 2012.0 
537 LUX Europe Western Europe 2013.0 
538 LUX Europe Western Europe 2014.0 
539 SRB Europe Southern Europe 2012.0 
540 SRB Europe Southern Europe 2013.0 
541 SRB Europe Southern Europe 2014.0 
542 MNE Europe Southern Europe 2012.0 
543 MNE Europe Southern Europe 2013.0 
544 MNE Europe Southern Europe 2014.0 
545 SDN Africa Northern Africa 2012.0 
546 SDN Africa Northern Africa 2013.0 
547 SDN Africa Northern Africa 2014.0 
548 SSD Africa Northern Africa 2012.0 
549 SSD Africa Northern Africa 2013.0 
550 SSD Africa Northern Africa 2014.0 
551 CHN Asia Eastern Asia 2012.0 
552 CHN Asia Eastern Asia 2013.0 
553 CHN Asia Eastern Asia 2014.0 

 population_c BiocapTotGHA_c EFConsTotGHA_c carbon_c \
0 2.978000e+06 2.230728e+06 6.371811e+06 2.601444e+06 
1 2.992000e+06 2.351551e+06 6.083715e+06 2.584959e+06 
2 3.006000e+06 2.362550e+06 6.068199e+06 2.560362e+06 
3 2.972700e+07 1.541892e+07 2.406439e+07 5.032811e+06 
4 3.068200e+07 1.558024e+07 2.425068e+07 5.112871e+06 
5 3.162800e+07 1.594269e+07 2.442466e+07 4.898568e+06 
6 2.881000e+06 3.084854e+06 6.426738e+06 2.790482e+06 
7 2.883000e+06 3.009000e+06 6.312022e+06 2.775526e+06 
8 2.890000e+06 3.038634e+06 6.188639e+06 2.592727e+06 
9 3.743900e+07 2.340175e+07 9.049695e+07 0.000000e+00 
10 3.818600e+07 2.380697e+07 9.476482e+07 0.000000e+00 
11 3.893400e+07 2.108411e+07 9.534057e+07 5.708592e+07 
12 2.268600e+07 5.338946e+07 2.621342e+07 9.555344e+06 
13 2.344800e+07 5.741712e+07 3.488198e+07 1.540105e+07 
14 2.422800e+07 5.548913e+07 3.776760e+07 1.876227e+07 
15 8.900000e+04 8.429075e+04 4.064051e+05 0.000000e+00 
16 9.000000e+04 8.461407e+04 3.751131e+05 0.000000e+00 
17 9.100000e+04 8.483893e+04 3.803687e+05 0.000000e+00 
18 4.209500e+07 2.777064e+08 1.382340e+08 5.524838e+07 
19 4.253800e+07 2.802609e+08 1.535348e+08 5.588603e+07 
20 4.298000e+07 2.883548e+08 1.585845e+08 5.505947e+07 
21 2.291100e+07 3.189218e+08 1.831576e+08 1.199016e+08 
22 2.327000e+07 3.071367e+08 1.714791e+08 1.167452e+08 
23 2.362200e+07 3.130207e+08 1.626579e+08 1.110269e+08 
24 8.455000e+06 2.473891e+07 5.034319e+07 3.454678e+07 
25 8.487000e+06 2.464946e+07 5.112553e+07 3.472088e+07 
26 8.51

Podemos ver los mejores hiperparámetros que para este ha encontrado

In [29]:
bestPipeline = cvModel.bestModel
bestModel = bestPipeline.stages[6]

print('numTrees - ', bestModel.getNumTrees)
print('maxDepth - ', bestModel.getOrDefault('maxDepth'))

numTrees - 30
maxDepth - 25